In [1]:
%matplotlib inline

import sys
sys.path.append('../')

from agentsClasses.OfflineDQNClass import OfflineDQNAgent, OfflineDDQNAgent
from utilities.plots import create_grids, plot_trajectory

In [2]:
#Hyperparameters
show_stats = True       #Mostrar stats de los agentes al terminar el entrenamiento
render = True           #Mostrar el grid con la política y trayectoria obtenidas

NUM_DQN_AGENTS = 1      #Numero de agentes DQN a entrenar
NUM_DDQN_AGENTS = 1     #Numero de agentes DDQN a entrenar

#Estructura de las redes neuronales de los agentes
NUM_NEURONS_FC1 = 128   #Neuronas de la primera capa fully connected
NUM_NEURONS_FC2 = 128   #Neuronas de la segunda primera capa fully connected

SHAPE = "5x5"           #Tamanho del grid "5x5" or "14x14"

if SHAPE == "5x5":
    EPISODES_PER_AGENT = 1000       #Episodios por agente
    MAX_STEPS_PER_EPISODE = 100     #Numero maximo de pasos por episodio
    FOLDER_NAME = "5x5_5_model"     #Carpeta donde se encuentre el ensemble que se quiere usar
    REWARD = [1000, -100, 0.3]      #[recompensa por alcanzar la meta, penalizacion por visitar estados por debajo de umbral de incertidumbre, umbral de incertidumbre para penalizar]

elif SHAPE == "14x14":
    EPISODES_PER_AGENT = 3000       #Episodios por agente
    MAX_STEPS_PER_EPISODE = 300     #Numero maximo de pasos por episodio
    FOLDER_NAME = "5_model"         #Carpeta donde se encuentre el ensemble que se quiere usar
    REWARD = [1000, -1, 0.3]        #[recompensa por alcanzar la meta, penalizacion por visitar estados por debajo de umbral de incertidumbre, umbral de incertidumbre para penalizar]

#Parametros de entrenamiento
EPS_START = 1.0
EPS_END = 0.01
EPS_DECAY = 0.9996      #En este caso la caida de epsilon es fija y no depende del numero de episodios

BUFFER_SIZE = int(1e5)  #Tamaño del buffer de repeticion de experiencias
BATCH_SIZE = 64         #Tamaño de minibatch
GAMMA = 0.99            #Factor de descuento
TAU = 1e-3              #Para realizar el soft update de los parametros target
LR = 5e-4               #Learning rate
UPDATE_EVERY = 4        #Cada cuantos pasos actualizar la red Q

In [ ]:
#CREATING AND TRAINING THE AGENTS
agents_arr = []         # array of agents

print("Starting training of", NUM_DQN_AGENTS, "DQN agents and", NUM_DDQN_AGENTS, "DDQN agents")

for i in range(NUM_DQN_AGENTS+NUM_DDQN_AGENTS):
    if i < NUM_DQN_AGENTS:
        print("DQN Agent", i+1,"/",NUM_DQN_AGENTS)    # if the agent is a DQN agent
        agent = OfflineDQNAgent(i, "DQN", SHAPE, FOLDER_NAME, NUM_NEURONS_FC1, NUM_NEURONS_FC2, EPISODES_PER_AGENT, MAX_STEPS_PER_EPISODE, REWARD, EPS_START, EPS_END, EPS_DECAY, BUFFER_SIZE, BATCH_SIZE, GAMMA, TAU, LR, UPDATE_EVERY)
    else:
        print("DDQN Agent", i+1-NUM_DQN_AGENTS,"/",NUM_DDQN_AGENTS)  # if the agent is a DDQN agent 
        agent = OfflineDDQNAgent(i, "DDQN", SHAPE, FOLDER_NAME, NUM_NEURONS_FC1, NUM_NEURONS_FC2, EPISODES_PER_AGENT, MAX_STEPS_PER_EPISODE, REWARD, EPS_START, EPS_END, EPS_DECAY, BUFFER_SIZE, BATCH_SIZE, GAMMA, TAU, LR, UPDATE_EVERY)

    agent.train()
    agents_arr.append(agent)

In [ ]:
if show_stats:
    for agent in agents_arr:
        print("La longitud estimada para el eje X del training error es:", (agent.total_steps/UPDATE_EVERY)*BATCH_SIZE)
        agent.plot_results(rolling_length=5, rolling_error=10)

In [ ]:
for agent in agents_arr:
    value_grid, policy_grid, string_policy_grid = create_grids(agent.env, Qnet=agent.qnetwork_local)
    start_pos = agent.env.unwrapped.start_pos

    if render:
        plot_trajectory(string_policy_grid, start_pos, agent.id+1)

In [ ]:
#Hay muchos elementos en el rewardHistory que son iguales, quiero que solo haya uno de cada
#historico = dict.fromkeys(agent.env.unwrapped.rewardHistory)

print(len(agent.env.unwrapped.rewardHistory))

unicos = []

for i in agent.env.unwrapped.rewardHistory:

    stringo = str(i[0][0]) + ',' + str(i[0][1]) + ',' + str(i[1]) + ',' + str(i[2][0]) + ',' + str(i[2][1])
    
    if stringo not in unicos:
        unicos.append(stringo)

for stringo in unicos:
    print(stringo)